In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
from scipy import stats
import pandas as pd
import flammkuchen as fl
from split_dataset import SplitDataset
#from bouter import Experiment
from scipy.interpolate import interp1d 
from scipy.signal import convolve2d
import colorspacious
import napari
import matplotlib.pyplot as plt

from skimage.filters import threshold_otsu
from fimpylab.core.twop_experiment import TwoPExperiment

from pathlib import Path

In [ ]:
master_hab = Path(r"Z:\Hagar and Ot\e0075\habenula")
fish_list_hab = list(master_hab.glob("*_f*"))
num_fish_hab = len(fish_list_hab)

master_ipn = Path(r"Z:\Hagar and Ot\e0075\ipn h2b")
fish_list_ipn = list(master_ipn.glob("*_f*"))
num_fish_ipn = len(fish_list_ipn)

In [ ]:
thresh = 0.2

In [ ]:
n_neurons_hab = np.zeros((num_fish_hab, 2))
n_tuned_neurons_hab = np.zeros((num_fish_hab, 2))

for fish in range(num_fish_hab):
    fish_path = fish_list_hab[fish]
    print(fish_path)
    suite2p_path = fish_path / 'suite2p'
    path_list = list(suite2p_path.glob("*00*"))

    count = 0
    #try:
    for path in path_list:
        #### get number of neurons in the left and right habenula for each fish
        habenula_coords = fl.load(path / 'habenula_coords.h5')
        lhb = habenula_coords['lhab_coords']
        rhb = habenula_coords['rhab_coords']

        n_neurons_hab[fish, 0] += len(lhb)
        n_neurons_hab[fish, 1] += len(rhb)

        rel_ind = fl.load(path / 'reliability_index_arr.h5')['reliability_arr_combined']

        if count is 0:
            rel_ind_lhb = rel_ind[lhb]
            rel_ind_rhb = rel_ind[rhb]
        else:
            rel_ind_lhb = np.append(rel_ind_lhb, rel_ind[lhb])
            rel_ind_rhb = np.append(rel_ind_rhb, rel_ind[rhb])
        count += 1

    #### reliability index for all neurons and get otsus threshold 
    rel_thresh = threshold_otsu(np.append(rel_ind_rhb, rel_ind_lhb))
    print("Reliability threshold: ", rel_thresh)

    ### get number of tuned neurons in the left and right habenula 
    n_tuned_neurons_hab[fish, 0] = np.shape(np.where(rel_ind_lhb > thresh)[0])[0]
    n_tuned_neurons_hab[fish, 1] = np.shape(np.where(rel_ind_rhb > thresh)[0])[0]
        
    #except:
    #    print('error')
    #    n_tuned_neurons_hab[fish, 0] = None
    #    n_tuned_neurons_hab[fish, 1] = None


In [ ]:
n_neurons_ipn = np.zeros((num_fish_ipn))
n_tuned_neurons_ipn = np.zeros((num_fish_ipn))

for fish in range(num_fish_ipn):
    fish_path = fish_list_ipn[fish]
    print(fish_path)
    suite2p_path = fish_path / 'suite2p'
    path_list = list(suite2p_path.glob("*00*"))

    count = 0
    for path in path_list:
        #### get number of neurons in the left and right habenula for each fish
        ipn_coords = fl.load(path / 'ipn_coords.h5')['coords_to_keep']
        
        rel_ind = fl.load(path / 'reliability_index_arr.h5')['reliability_arr_combined'][ipn_coords]
        #n_neurons_ipn[fish] += len(ipn_coords)
        
        if count is 0 or rel_ind_ipn is 0:
            rel_ind_ipn = rel_ind
        else:
            rel_ind_ipn = np.append(rel_ind_ipn, rel_ind)
        count += 1

    #### reliability index for all neurons and get otsus threshold 
    rel_thresh = threshold_otsu(rel_ind_ipn)
    print("Reliability threshold: ", rel_thresh)

    ### get number of tuned neurons in the left and right habenula 
    n_neurons_ipn[fish] = np.shape(rel_ind_ipn)[0]
    print(n_neurons_ipn[fish])
    n_tuned_neurons_ipn[fish] = np.shape(np.where(rel_ind_ipn > thresh)[0])[0]
        


In [ ]:
np.shape(rel_ind)[0]

In [ ]:
frac_tuned_neurons_hab

In [ ]:
frac_tuned_neurons_hab = n_tuned_neurons_hab / n_neurons_hab * 100

In [ ]:
frac_tuned_neurons_ipn = n_tuned_neurons_ipn / n_neurons_ipn * 100

In [ ]:
### plot fraction of neurons in each region - overll and for each direction 
fig, ax = plt.subplots(1,1, figsize=(3,3))

ax.scatter(np.zeros(num_fish_hab), frac_tuned_neurons_hab[:,0], color='lightgray')
ax.errorbar([0], np.nanmean(frac_tuned_neurons_hab[:,0]), np.nanstd(frac_tuned_neurons_hab[:,0]) / num_fish_hab, color='k')
#ax.violinplot(frac_tuned_neurons, [0,1])
ax.scatter(np.ones(num_fish_hab), frac_tuned_neurons_hab[:,1], color='lightgray')
ax.errorbar([1], np.nanmean(frac_tuned_neurons_hab[:,1]), np.nanstd(frac_tuned_neurons_hab[:,1]) / num_fish_hab, color='k')

ax.scatter([0, 1], np.nanmean(frac_tuned_neurons_hab, axis=0), color='k')

ax.set_ylim(0, 100)
ax.set_xlim(-0.5, 2.5)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

In [ ]:
### plot fraction of neurons in each region - overll and for each direction 
ax.scatter(np.ones(num_fish_ipn)*2, frac_tuned_neurons_ipn, color='lightgray')
ax.errorbar([2], np.nanmean(frac_tuned_neurons_ipn), np.nanstd(frac_tuned_neurons_ipn) / num_fish_ipn, color='k')

ax.scatter([2], np.nanmean(frac_tuned_neurons_ipn), color='k')

In [ ]:
fig.savefig(master_hab / 'fraction of reliable neurons in habenula and ipn thresh 0.2.pdf', dpi=300)

In [ ]:
##### fraction of tuned neurons for each position 

In [ ]:
n_tuned_neurons_ipn

In [ ]:
n_neurons_ipn

In [ ]:
frac_tuned_neurons_ipn

In [ ]:
fish = 1
fish_path = fish_list_ipn[fish]
print(fish_path)
suite2p_path = fish_path / 'suite2p'
path_list = list(suite2p_path.glob("*00*"))
print(path_list)
count = 0
for path in path_list:
    print(path)
    #### get number of neurons in the left and right habenula for each fish
    ipn_coords = fl.load(path / 'ipn_coords.h5')['coords_to_keep']

    rel_ind = fl.load(path / 'reliability_index_arr.h5')['reliability_arr_combined'][ipn_coords]
    #n_neurons_ipn[fish] += len(ipn_coords)

    if count is 0 or rel_ind_ipn is 0:
        rel_ind_ipn = rel_ind
    else:
        rel_ind_ipn = np.append(rel_ind_ipn, rel_ind)
    count += 1

#### reliability index for all neurons and get otsus threshold 
rel_thresh = threshold_otsu(rel_ind_ipn)
print("Reliability threshold: ", rel_thresh)

### get number of tuned neurons in the left and right habenula 
n_neurons_ipn[fish] = np.shape(rel_ind_ipn)[0]
print(n_neurons_ipn[fish])
n_tuned_neurons_ipn[fish] = np.shape(np.where(rel_ind_ipn > thresh)[0])[0]
        